In [1]:
#pip install nextrec

In [2]:
import os
import sys
import logging
os.environ["TOKENIZERS_PARALLELISM"]="false"

logger = logging.getLogger() 
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)s %(message)s'))
logger.handlers = [handler] 

在这个示例里，我们将指导您使用NextRec框架，利用RQ-VAE模型训练和生成语义ID。在`dataset/`路径下，我们为您提供了示例电商数据集，其中包含了用户id，物品id，商品文本描述，埋点时间和一些常见的特征。我将会用到其中的文本特征来进行训练。

In [ ]:
import pandas as pd

df = pd.read_csv('/NextRec/dataset/ecommerce_task.csv')
df.head()

,log_time,user_id,item_id,gender,age_bucket,city,device,channel,category,brand,price,impression_position,user_active_days_7,user_ctr,text_desc,click,conversion
0,2025-11-14 09:16:00,10001,item_BEAU_01399,F,35-44,Beijing,Web,search,Beauty,PureSkin,277.62,10,7,0.29,Sunscreen cruelty-free brightening,0,0
1,2025-11-15 17:18:00,10001,item_BEAU_01530,F,35-44,Beijing,Web,push,Beauty,BrandN,814.57,9,7,0.29,Essence for sensitive skin with niacinamide hy...,0,0
2,2025-11-18 09:51:00,10001,item_HOME_01211,F,35-44,Beijing,Web,organic,Home,BrandB,904.47,5,7,0.29,Office chair with lumbar support handcrafted,0,0
3,2025-11-23 09:59:00,10001,item_CLOT_00426,F,35-44,Beijing,Web,search,Clothing,BrandD,358.71,10,7,0.29,Cotton t-shirt short sleeve athletic fit with ...,1,1
4,2025-12-08 19:39:00,10001,item_HOME_01178,F,35-44,Beijing,Web,search,Home,ModernLiving,315.23,10,7,0.29,Plant pot space-saving durable material easy a...,0,0


# RQ-VAE

在开始之前，先简单介绍一下RQ-VAE。RQ-VAE是对VAE（变分自编码器）的改进，后者是一个生成模型，目标是通过学习输入数据的分布，输出类似的分布，使用KL散度作为评估指标。简单来说，VAE类似于一个embedding层，以文本数据为例，将其映射到低维空间。由于VAE需要输入一个向量，因此需要对文本进行分词和嵌入/one hot，这样每个token都会得到一个高维度向量，随后将这个高维序列传入编码器（RNN/GRN/CNN/Transformer），来将序列压缩为低维向量，这就是最终需要的低维向量。

VAE已经将输入的高维embedding降低为了一定程度上的低维embedding，不过要对所有item都保存这个向量，对工业的数据存储压力依旧很大，因此希望对这个输出再进行一次压缩，将连续向量压缩为离散低维向量，这就是RQ-VAE试图解决的问题。这里的RQ指的是向量量化（Vector Quantization, VQ）和残差编码（Residual Encoding）。RQ-VAE试图对Item保存为索引ID，计算时通过ID → 查表 → 点积的方式进行召回。

RQ-VAE通过离散码本（Codebook）索引的组合向量来表示原来的VAE embedding。

In [4]:
from nextrec.basic.features import DenseFeature
from nextrec.data.dataloader import RecDataLoader
from nextrec.models.representation import RQVAE

from nextrec.data.data_processing import split_dict_random
from nextrec.utils.embedding import encode_multimodel_content

要通过RQ-VAE生成sid，首先就需要输入的多模态信息，在这个示例里，我们采用文本信息作为原始输出，使用bert模型将其嵌入为高维稠密向量。NextRec提供的工具函数`encode_multimodel_content`可以帮你实现这一步。它底层调用了transformers库来进行嵌入。

In [5]:
pip show nextrec

Name: nextrec
Version: 0.4.21
Summary: A comprehensive recommendation library with match, ranking, and multi-task learning models
Home-page: https://github.com/zerolovesea/NextRec
Author: Yang Zhou
Author-email: zyaztec@gmail.com
License: 
Location: /opt/anaconda3/envs/nextrec/lib/python3.10/site-packages
Editable project location: /Users/zyaztec/DailyWork/建模代码整理/NextRec
Requires: numpy, pandas, pyarrow, pyyaml, rich, scikit-learn, scipy, swanlab, torch, torchvision, transformers, wandb
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [6]:
texts = df["text_desc"].fillna("").tolist()
print(f"Dataset loaded: {len(df)} samples")

embeddings = encode_multimodel_content(texts, model_name="bert-base-uncased", device="cpu", batch_size=32)
print(f"Encoded text_desc into embeddings with shape {embeddings.shape}")
print(embeddings[0])

Dataset loaded: 10000 samples
Encoded text_desc into embeddings with shape torch.Size([10000, 768])
tensor([-6.9366e-02, -2.5368e-01, -4.2544e-01,  2.4386e-01, -2.3643e-01,
        -2.3092e-02, -7.0698e-02,  2.4566e-01, -4.0289e-02, -1.3284e-01,
         9.2764e-02,  4.7631e-01, -2.7675e-02,  1.6549e-01, -2.5294e-01,
         1.0878e-01, -2.4900e-01,  4.8581e-01,  3.5214e-02,  1.4288e-01,
        -8.4659e-02, -8.7917e-02, -6.5080e-01,  1.9214e-01,  9.4129e-02,
        -1.0084e-01,  3.9686e-01,  2.1918e-01,  1.7440e-01,  1.8361e-01,
         2.5563e-01,  1.7434e-01, -5.5120e-02, -5.0393e-02,  5.2894e-01,
        -5.3674e-01,  2.0702e-02, -8.3346e-01, -2.1497e-01, -9.4487e-02,
        -1.1800e-02,  1.2713e-01,  6.3430e-01, -1.8029e-01,  5.2011e-02,
        -3.1240e-01, -2.5302e+00,  2.3571e-01, -1.6146e-01, -1.8182e-01,
         6.8951e-01, -4.4509e-01,  6.4377e-02,  1.4384e-01,  6.2194e-01,
         8.5790e-01, -2.3083e-02,  5.3937e-01,  3.3104e-01,  1.1583e-01,
         8.1912e-02, -3.

现在我们拥有原始输入了，我们需要让RQ-VAE来学习和重建输入数据的分布。因此，我们需要先为原始数据拆分成训练和验证集，并构建成dataloader。NextRec提供了`RecDataLoader`来帮助你实现这一步。

由于我们已经将文本特征变为了稠密特征`Dense Feature`，我们只需要对特征进行定义，随后传入`RecDataLoader`即可。未来NextRec将会支持多模态特征定义和自动transform，不过在此之前，您还需要使用工具函数来进行手动transform。

In [7]:
# Build loaders for RQ-VAE
text_feature = DenseFeature(name="text_embedding", input_dim=embeddings.shape[1])
loader_builder = RecDataLoader(dense_features=[text_feature])
emb_np = embeddings.cpu().numpy()
rqvae_train_dict, rqvae_valid_dict = split_dict_random(
    {"text_embedding": emb_np}, test_size=0.1, random_state=2025
)
rqvae_train_loader = loader_builder.create_dataloader(
    rqvae_train_dict, batch_size=256, shuffle=True
)
rqvae_valid_loader = loader_builder.create_dataloader(
    rqvae_valid_dict, batch_size=256, shuffle=False
)
rqvae_full_loader = loader_builder.create_dataloader(
    {"text_embedding": emb_np}, batch_size=256, shuffle=False
)

现在我们需要实例化RQ-VAE，可以看到它提供了多个参数，我们将为您一一解释：

- input_dim: 输入嵌入的维度（如 BERT 向量 768）；编码器接收的维度，解码器最终输出也回到该维度。
- hidden_dims: 编码器/解码器的中间层维度列表。
- latent_dim: 编码后潜在空间及码本向量的维度；越小压缩越强，越大表达力更高。
- num_codebooks: 残差量化层数（分层深度）；层数越多语义 ID 粒度越细，但推理/训练稍慢。
- codebook_size: 每一层码本的词表大小列表，长度应等于 num_codebooks；例如 [256,256,256] 意味 3 层，每层 256 个码字，总组合 256³。
- shared_codebook: 是否让所有层共享同一套码本；共享码本时参数量更少，不共享时每层的表达力更强，默认为不共享。
- kmeans_method: 码本初始化方式；"kmeans" 常规 KMeans，"bkmeans" 平衡 KMeans（推荐），其他值则随机初始化。
- kmeans_iters: KMeans 初始化的最大迭代次数。
- distances_method: 量化距离度量；"l2" 欧氏距离（默认 VAE 用法），此外也支持"cosine" 。
- loss_beta: 承诺损失权重 β（量化损失中的第二项）；越大越强制编码器贴近码本，通常 0.25 左右。
- dense_features: RQ-VAE需要传入的原始embedding分布是稠密向量，我们需要告诉模型哪些向量是需要被学习和压缩的。

在配置完成后，我们使用`fit`方法来开始训练。与其他精排模型的`fit`不同，RQ-VAE需要配置`init_batches`参数来指定使用多少批次的数据进行码本初始化。

In [8]:
rqvae = RQVAE(
    input_dim=embeddings.shape[1],
    hidden_dims=[128, 256],
    latent_dim=128,
    num_codebooks=2,
    codebook_size=[128, 128],
    shared_codebook=False,
    kmeans_method="bkmeans",
    kmeans_iters=50,
    distances_method="cosine",
    loss_beta=0.25,
    device="cpu",
    dense_features=[DenseFeature(name="text_embedding", input_dim=embeddings.shape[1])],
    session_id="rqvae_tutorial",
)
rqvae.fit(
    train_data=rqvae_train_loader,
    valid_data=rqvae_valid_loader,
    epochs=5,
    batch_size=256,
    lr=1e-3,
    init_batches=3,
)


Model Summary: RQVAE


Feature Configuration
--------------------------------------------------------------------------------
Dense Features (1):
  1. text_embedding      

Model Parameters
--------------------------------------------------------------------------------
Model Architecture:
RQVAE(
  (encoder): RQEncoder(
    (stages): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=768, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): Sequential(
        (0): Linear(in_features=128, out_features=256, bias=True)
        (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (2): Linear(in_features=256, out_features=128, bias=True)
    )
  )
  (decoder): RQDecoder(
    (stages): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=128, out_features=256, bias=True)
        (1): Bat

Epoch 1/5: 36/36 elapsed=0:00:00 speed=108.30/s ETA=0:00:00


Epoch 1/5 - Train Loss: 0.3739
  Epoch 1/5 - Valid Loss: 0.1670


Epoch 2/5: 36/36 elapsed=0:00:00 speed=103.61/s ETA=0:00:00


Epoch 2/5 - Train Loss: 0.1144
  Epoch 2/5 - Valid Loss: 0.0847


...

Epoch 4/5: 36/36 elapsed=0:00:00 speed=120.19/s ETA=0:00:00


Epoch 4/5 - Train Loss: 0.0621
  Epoch 4/5 - Valid Loss: 0.0598


Epoch 5/5: 36/36 elapsed=0:00:00 speed=123.27/s ETA=0:00:00


Epoch 5/5 - Train Loss: 0.0609
  Epoch 5/5 - Valid Loss: 0.0624

Training finished.



RQVAE(
  (encoder): RQEncoder(
    (stages): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=768, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): Sequential(
        (0): Linear(in_features=128, out_features=256, bias=True)
        (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (2): Linear(in_features=256, out_features=128, bias=True)
    )
  )
  (decoder): RQDecoder(
    (stages): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=128, out_features=256, bias=True)
        (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): Sequential(
        (0): Linear(in_features=256, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

现在我们已经训练完成了，让我们简单看一下输出的结果。可以看到，每个样本原本768维的连续嵌入向量，现在只用2维的离散向量即可表达，大大节省了计算量和存储量。

In [9]:
semantic_ids = rqvae.predict(
    rqvae_full_loader, batch_size=256, return_reconstruction=False, as_numpy=False
)
semantic_ids = semantic_ids.to("cpu")
print(f"Semantic IDs shape: {semantic_ids.shape}")
print(f"Semantic IDs sample (first 5 items):\n{semantic_ids[:5]}")

Semantic IDs shape: torch.Size([10000, 2])
Semantic IDs sample (first 5 items):
tensor([[ 72,  30],
        [ 72,  52],
        [106,  38],
        [  0,  38],
        [ 28, 101]])
